# **미션**
```markdown
https://corners.gmarket.co.kr/BestSellers 에서
상품 코드, 상세 페이지 url, 이미지 url, 할인 후 가격 수집. 만약 할인 후 가격이 없으면 원래 가격 수집
```

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

GMARKET_PRODUCT_URL="https://corners.gmarket.co.kr/BestSellers"
response = requests.get(GMARKET_PRODUCT_URL)
html_code = response.content
soup = BeautifulSoup(html_code, 'html.parser')

# 상세페이지 url 수집
page_url_lst=[]
for i in range(200):
  page_url_lst.append(soup.select("a.itemname")[i]["href"]) 

#이미지 url 수집
product_url_list=[]
for i in range(200):
  product_url_list.append('https:'+soup.select("div.thumb > a > .lazy")[i]["data-original"])

#상품코드 수집
product_code_list=[]
for i in range(200):
    if "'" in soup.select("a.itemname")[i]["onclick"].split("'}")[0][-10:]:
        product_code_list.append(soup.select("a.itemname")[i]["onclick"].split("'}")[0][-10:].replace("'",""))
    else:
        product_code_list.append(soup.select("a.itemname")[i]["onclick"].split("'}")[0][-10:])

#가격 수집
price_list=[]
for i in range(200):
  if "할인가" not in soup.select(".item_price")[i].text:
    price_list.append(soup.select(".item_price > .o-price")[i].text.split('정가')[1])
  elif "할인가" in soup.select(".item_price")[i].text:
    price_list.append(soup.select(".item_price > .s-price > strong")[i].text)

data={"상품코드":product_code_list,"상세 페이지 url":page_url_lst,"이미지 url":product_url_list,"가격":price_list}
product_data=pd.DataFrame(data)

product_data.to_csv('G마켓 베스트 상품 크롤링.csv')

# 보완 
- 정규표현식 사용하면 상품코드, 가격에서 "숫자만" 추출 가능 
> 정규표현식 공부하기
- 오류 : 9글자 짜리 상품코드
: 지금 코드에서 9자리 상품코드를 살릴 수 있는 방법 
>if조건문 사용해서 (') 포함되어 있으면 제거하게끔 해보자

In [3]:
soup.select("a.itemname")[i]["onclick"].split("'}")[0][-10:]

'2249842803'

In [4]:
product_code_list=[]
for i in range(200):
    
    if "'" in soup.select("a.itemname")[i]["onclick"].split("'}")[0][-10:]:
        product_code_list.append(soup.select("a.itemname")[i]["onclick"].split("'}")[0][-10:].replace("'",""))
    else:
        product_code_list.append(soup.select("a.itemname")[i]["onclick"].split("'}")[0][-10:])

In [5]:
product_data

,상품코드,상세 페이지 url,이미지 url,가격
0,1592768819,http://item.gmarket.co.kr/Item?goodscode=15927...,https://gdimg.gmarket.co.kr/1592768819/still/3...,"14,900원"
1,2810927407,http://item.gmarket.co.kr/Item?goodscode=28109...,https://gdimg.gmarket.co.kr/2810927407/still/3...,"18,900원"
2,1833766346,http://item.gmarket.co.kr/Item?goodscode=18337...,https://gdimg.gmarket.co.kr/1833766346/still/3...,"16,900원"
3,2388032148,http://item.gmarket.co.kr/Item?goodscode=23880...,https://gdimg.gmarket.co.kr/2388032148/still/3...,"17,900원"
4,1707731465,http://item.gmarket.co.kr/Item?goodscode=17077...,https://gdimg.gmarket.co.kr/1707731465/still/3...,"14,940원"
...,...,...,...,...
195,400871660,http://item.gmarket.co.kr/Item?goodscode=40087...,https://gdimg.gmarket.co.kr/400871660/still/30...,"15,000원"
196,173366583,http://item.gmarket.co.kr/Item?goodscode=17336...,https://gdimg.gmarket.co.kr/173366583/still/30...,"29,000원"
197,2123055394,http://item.gmarket.co.kr/Item?goodscode=21230...,https://gdimg.gmarket.co.kr/2123055394/still/3...,"30,900원"
198,203771527,http://item.gmarket.co.kr/Item?goodscode=20377...,https://gdimg.gmarket.co.kr/203771527/still/30...,"45,900원"
